In [3]:
from braindecode.datasets.moabb import MOABBDataset

#subject_id = [1,2,3,4,5,6,7,8,9]
subject_id = [1]
#2a= BNCI2014001 2b= BNCI2014004
dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=subject_id)
print(dataset)


In [28]:
dataset.datasets[4]
dataset.save("dataset\\2b_moabb\\train")

Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\0-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\0-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\1-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\1-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\2-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\2-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\3-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\3-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\4-raw.fif
Closing c:\Users\asus\Desktop\Motorimagery_for_gamification\dataset\2b_moabb\train\4-raw.fif
[done]
Writing c:\Users\asus\Desktop\Motor

Opening raw data file <braindecode.datasets.moabb.MOABBDataset object at 0x0000018289D56940>...


TypeError: fname must be an instance of path-like, got <class 'braindecode.datasets.moabb.MOABBDataset'> instead.